In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import os
import sys
import datetime
import joblib

In [3]:
# --- 1. Configurações de Ficheiros ---
BASE_DRIVE_PATH = "/content/drive/MyDrive"

TRAINING_DATA_FILE = os.path.join(BASE_DRIVE_PATH, "training_data.csv")

DNABERT_EMBEDDINGS_FILE = os.path.join(BASE_DRIVE_PATH, "dnabert_embeddings.npy")

NORMALIZATION_SCALER_FILE = os.path.join(BASE_DRIVE_PATH, "scaler_standard.pkl")

RESULTS_OUTPUT_DIR = BASE_DRIVE_PATH
os.makedirs(RESULTS_OUTPUT_DIR, exist_ok=True)

RESULTS_FILE_PREFIX = "svm_standard_optimization_results"
PREDICTIONS_DATA_PREFIX = "svm_standard_predictions_data"

In [4]:
# --- 2. Carregar Dados Originais e Embeddings Pré-Calculados ---

try:
    df = pd.read_csv(TRAINING_DATA_FILE)
    print(f"Dados originais carregados com sucesso de {TRAINING_DATA_FILE}. Dimensão: {df.shape}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro de dados original '{TRAINING_DATA_FILE}' não foi encontrado.")
    sys.exit(1)

if not all(col in df.columns for col in ['target', 'sequence', 'prot_z']):
    print("ERRO: Colunas inesperadas no DataFrame original. Certifique-se de que as colunas 'target', 'sequence', e 'prot_z' existem.")
    sys.exit(1)

y = df['prot_z'].values.ravel()

scaler = None
try:
    scaler = joblib.load(NORMALIZATION_SCALER_FILE)
    print(f"StandardScaler carregado com sucesso de: {NORMALIZATION_SCALER_FILE}")
    print(f"Média do scaler para desestandardização: {scaler.mean_[0]:.4f}, Desvio Padrão: {scaler.scale_[0]:.4f}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro StandardScaler '{NORMALIZATION_SCALER_FILE}' não foi encontrado.")
    print("É VITAL ter este ficheiro para desestandardizar as previsões corretamente.")
    print("Por favor, garanta que 'scaler_standard.pkl' está no diretório 'results/' ou ajuste o caminho.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO: Falha ao carregar StandardScaler de {NORMALIZATION_SCALER_FILE}: {e}")
    sys.exit(1)

true_original_target_values = scaler.inverse_transform(df['prot_z'].values.reshape(-1, 1)).flatten()
print(f"Verdadeiros valores originais do target (para métricas) gerados. Exemplo (primeiros 5): {true_original_target_values[:5]}")

try:
    X = np.load(DNABERT_EMBEDDINGS_FILE)
    if X.shape[0] != len(df):
        print(f"AVISO: O número de amostras nos embeddings ({X.shape[0]}) não corresponde ao número de amostras no CSV ({len(df)}).")
        print("Pode indicar uma inconsistência nos dados. Prosseguindo, mas verifique a fonte dos embeddings.")
    print(f"Embeddings DNABERT carregados com sucesso de {DNABERT_EMBEDDINGS_FILE}. Dimensão: {X.shape}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro de embeddings '{DNABERT_EMBEDDINGS_FILE}' não foi encontrado.")
    print(f"Por favor, certifique-se de que o ficheiro .npy com os embeddings está no caminho correto e acessível.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO: Falha ao carregar o ficheiro de embeddings '{DNABERT_EMBEDDINGS_FILE}': {e}")
    sys.exit(1)

print(f"\nDimensão de X (features): {X.shape}")
print(f"Dimensão de y (target 'prot_z' JÁ ESTANDARDIZADO): {y.shape}")

Dados originais carregados com sucesso de /content/drive/MyDrive/training_data.csv. Dimensão: (9336, 4)
StandardScaler carregado com sucesso de: /content/drive/MyDrive/scaler_standard.pkl
Média do scaler para desestandardização: 61077.1584, Desvio Padrão: 64770.3892
Verdadeiros valores originais do target (para métricas) gerados. Exemplo (primeiros 5): [1410.9918352  1443.35117936 1559.28121553 1501.84533491 1680.76889704]


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Embeddings DNABERT carregados com sucesso de /content/drive/MyDrive/dnabert_embeddings.npy. Dimensão: (9336, 768)

Dimensão de X (features): (9336, 768)
Dimensão de y (target 'prot_z' JÁ ESTANDARDIZADO): (9336,)


In [5]:
# --- 3. Treinar o SVR ---
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

param_distributions = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'gamma': ['scale', 'auto', 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
    'kernel': ['rbf']
}

In [6]:
# --- 3. Treinar o SVM (SVR - Support Vector Regressor) ---
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, KFold

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

param_distributions = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'gamma': ['scale', 'auto', 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
    'kernel': ['rbf']
}

svr = SVR()

print(f"\n--- Iniciando otimização de hiperparâmetros com HalvingRandomSearchCV ---")
print(f"Utilizando {n_splits} folds de validação cruzada.")
print("Este processo utiliza múltiplos núcleos de CPU e é mais eficiente que o RandomizedSearchCV completo.")

random_search = HalvingRandomSearchCV(
    estimator=SVR(),
    param_distributions=param_distributions,
    factor=3,
    resource='n_samples',
    min_resources=500,
    max_resources=9000,
    n_candidates=50,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    random_state=42
)


random_search.fit(X, y)

best_svr_model = random_search.best_estimator_


--- Iniciando otimização de hiperparâmetros com HalvingRandomSearchCV ---
Utilizando 5 folds de validação cruzada.
Este processo utiliza múltiplos núcleos de CPU e é mais eficiente que o RandomizedSearchCV completo.
n_iterations: 3
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 500
max_resources_: 9000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 50
n_resources: 500
Fitting 5 folds for each of 50 candidates, totalling 250 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [7]:
# --- 4. Avaliar o Modelo ---
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    best_svr_model.fit(X_train, y_train)
    y_pred_standardized = best_svr_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred_standardized)
    mae = mean_absolute_error(y_test, y_pred_standardized)
    r2 = r2_score(y_test, y_pred_standardized)

    try:
        pearson_corr, _ = pearsonr(y_test, y_pred_standardized)
    except ValueError:
        pearson_corr = np.nan

    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)
    pearson_scores.append(pearson_corr)

    print(f"Fold {fold+1}: MSE={mse:.4f}, MAE={mae:.4f}, R2={r2:.4f}, Pearson={pearson_corr:.4f}")

print("\n--- Resultados Finais da Validação Cruzada ---")
final_cv_results = {
    "MSE médio (escala estandarizada)": f"{np.nanmean(mse_scores):.4f} +/- {np.nanstd(mse_scores):.4f}",
    "MAE médio (escala estandarizada)": f"{np.nanmean(mae_scores):.4f} +/- {np.nanstd(mae_scores):.4f}",
    "R2 médio (escala estandarizada)": f"{np.nanmean(r2_scores):.4f} +/- {np.nanstd(r2_scores):.4f}",
    "Correlação de Pearson média (escala estandarizada)": f"{np.nanmean(pearson_scores):.4f} +/- {np.nanstd(pearson_scores):.4f}"
}
for metric, value in final_cv_results.items():
    print(f"{metric}: {value}")

Fold 1: MSE=0.4183, MAE=0.4745, R2=0.6028, Pearson=0.7782
Fold 2: MSE=0.3761, MAE=0.4515, R2=0.6118, Pearson=0.7829
Fold 3: MSE=0.3941, MAE=0.4658, R2=0.6132, Pearson=0.7847
Fold 4: MSE=0.3929, MAE=0.4602, R2=0.5951, Pearson=0.7725
Fold 5: MSE=0.3666, MAE=0.4497, R2=0.6278, Pearson=0.7927

--- Resultados Finais da Validação Cruzada ---
MSE médio (escala estandarizada): 0.3896 +/- 0.0177
MAE médio (escala estandarizada): 0.4603 +/- 0.0092
R2 médio (escala estandarizada): 0.6101 +/- 0.0110
Correlação de Pearson média (escala estandarizada): 0.7822 +/- 0.0068


In [8]:
# --- 5. Avaliação Final no Dataset Completo ---

final_model_for_full_data = random_search.best_estimator_
final_model_for_full_data.fit(X, y)
final_predictions_standardized = final_model_for_full_data.predict(X)

final_predictions_original = scaler.inverse_transform(final_predictions_standardized.reshape(-1, 1)).flatten()

global_mse = mean_squared_error(true_original_target_values, final_predictions_original)
global_mae = mean_absolute_error(true_original_target_values, final_predictions_original)
global_r2 = r2_score(true_original_target_values, final_predictions_original)


try:
    global_pearson, _ = pearsonr(true_original_target_values.flatten(), final_predictions_original.flatten())
except ValueError:
    global_pearson = np.nan

global_rmse = np.sqrt(global_mse)

mean_of_original_targets = np.mean(true_original_target_values)

if mean_of_original_targets != 0:
    pmae = (global_mae / mean_of_original_targets) * 100
    prmse = (global_rmse / mean_of_original_targets) * 100
else:
    pmae = np.nan
    prmse = np.nan


global_results = {
    "MSE Global (escala original)": f"{global_mse:.4f}",
    "MAE Global (escala original)": f"{global_mae:.4f}",
    "MAE Percentual (vs. Média)": f"{pmae:.2f}%",
    "RMSE Global (escala original)": f"{global_rmse:.4f}",
    "RMSE Percentual (vs. Média)": f"{prmse:.2f}%",
    "R2 Global (escala original)": f"{global_r2:.4f}",
    "Pearson Global (escala original)": f"{global_pearson:.4f}"
}
for metric, value in global_results.items():
    print(f"{metric}: {value}")


min_original_prot = np.min(true_original_target_values)
max_original_prot = np.max(true_original_target_values)
mean_original_prot = np.mean(true_original_target_values)
std_original_prot = np.std(true_original_target_values)
amplitude_original_prot = max_original_prot - min_original_prot

print(f"\n--- Escala dos Valores Originais de Expressão Proteica ---")
print(f"  Mínimo: {min_original_prot:.4f}")
print(f"  Máximo: {max_original_prot:.4f}")
print(f"  Média: {mean_of_original_targets:.4f}")
print(f"  Desvio Padrão: {std_original_prot:.4f}")
print(f"  Amplitude: {amplitude_original_prot:.4f}")

MSE Global (escala original): 600864490.6510
MAE Global (escala original): 15066.7422
MAE Percentual (vs. Média): 24.67%
RMSE Global (escala original): 24512.5374
RMSE Percentual (vs. Média): 40.13%
R2 Global (escala original): 0.8568
Pearson Global (escala original): 0.9293

--- Escala dos Valores Originais de Expressão Proteica ---
  Mínimo: 1357.1507
  Máximo: 204059.9524
  Média: 61077.1584
  Desvio Padrão: 64770.3892
  Amplitude: 202702.8018


In [9]:
# --- 6. Guardar Resultados ---
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
results_filename = os.path.join(RESULTS_OUTPUT_DIR, f"{RESULTS_FILE_PREFIX}_{timestamp}.txt")
predictions_data_filename = os.path.join(RESULTS_OUTPUT_DIR, f"{PREDICTIONS_DATA_PREFIX}_{timestamp}.npy")
model_save_filename = os.path.join(RESULTS_OUTPUT_DIR, f"best_svr_model_{timestamp}.joblib")


print(f"\n--- A guardar resultados em: {results_filename} ---")
with open(results_filename, 'w') as f:
    f.write(f"Resultados da Otimização de Hiperparâmetros do SVM\n")
    f.write(f"Data e Hora: {timestamp}\n")
    f.write(f"----------------------------------------------------\n\n")

    f.write(f"Melhores Hiperparâmetros Encontrados:\n")
    for param, value in random_search.best_params_.items():
        f.write(f"  {param}: {value}\n")
    f.write(f"\n")

    f.write(f"Resultados da Validação Cruzada (Médias e Desvios Padrão - ESCALA ESTANDARDIZADA):\n")
    for metric, value in final_cv_results.items():
        f.write(f"  {metric}: {value}\n")
    f.write(f"\n")

    f.write(f"Métricas no Dataset Completo (Com o melhor modelo treinado uma vez no dataset inteiro - ESCALA ORIGINAL):\n")
    for metric, value in global_results.items():
        f.write(f"  {metric}: {value}\n")
    f.write(f"\n")
    f.write(f"Informações Adicionais:\n")
    f.write(f"  Número de Amostras: {X.shape[0]}\n")
    f.write(f"  Número de Features: {X.shape[1]}\n")
    f.write(f"  Tipo de Embeddings: DNABERT\n")
    f.write(f"  Folds de Validação Cruzada: {n_splits}\n")
    f.write(f"  Ficheiro de Embeddings Usado: {os.path.basename(DNABERT_EMBEDDINGS_FILE)}\n")
    f.write(f"  Ficheiro de Modelo SVR Guardado: {os.path.basename(model_save_filename)}\n")
    f.write(f"  Ficheiro de Dados de Previsões Guardado: {os.path.basename(predictions_data_filename)}\n")
    f.write(f"  Ficheiro de StandardScaler Guardado: {os.path.basename(NORMALIZATION_SCALER_FILE)}\n")


print(f"Resultados guardados com sucesso em {results_filename}.")

data_for_plotting = np.vstack((true_original_target_values.flatten(), final_predictions_original)).T
np.save(predictions_data_filename, data_for_plotting)
print(f"Valores reais (escala original) e previstos (escala original) guardados em {predictions_data_filename} para análise de gráficos.")

joblib.dump(best_svr_model, model_save_filename)
print(f"Melhor modelo SVR guardado em: {model_save_filename}")

print("\nScript de otimização de SVM concluído.")


--- A guardar resultados em: /content/drive/MyDrive/svm_standard_optimization_results_20250620_233210.txt ---
Resultados guardados com sucesso em /content/drive/MyDrive/svm_standard_optimization_results_20250620_233210.txt.
Valores reais (escala original) e previstos (escala original) guardados em /content/drive/MyDrive/svm_standard_predictions_data_20250620_233210.npy para análise de gráficos.
Melhor modelo SVR guardado em: /content/drive/MyDrive/best_svr_model_20250620_233210.joblib

Script de otimização de SVM concluído.
